# Analyzing Healthy UC Berkeley Dining Hall Meals in Comparison to the Mediterranean Diet

Contributors: Contributors: Richa Bhattacharya, Audrey Booher, Anoushka Gupta, Daysha Patalot, Yinan Xia


### <u> Goal of this study </u> ###

<b> [A] Description of Population of Interest: <b> 
    
Our population of interest is the UC Berkeley undergraduate student population. Freshmen, who live on campus, are designated a mandatory meal plan, while other undergraduate students may opt in to a meal plan, dine at campus restaurants, or cook for themselves at home. The question our study addresses is, as a UC Berkeley student, should one buy a meal plan or cook for oneself and abide by the Mediterranean diet if one wishes to maintain a subsistence diet at the lowest cost. 

### <u> Context </u>

<i> What is the Mediterranean Diet </i>

The Mediterranean diet is based on the traditional diet of the Mediterranean region, including Spain, Italy, and Greece. These regions have exceptionally low rates of chronic diseases and are rather healthy, and so the Mediterranean diet has been studied for its benefits. This diet is believed to support brain function, promote heart health, regulate blood sugar levels,  lower blood pressure, etc. The diet recommends eating nutritious fruits, vegetables, nuts, legumes, whole grains, seafood, and potatoes, etc, while moderating the intake of eggs, cheese, poultry and yogurt, and rarely consuming red meat, processed foods, foods with added sugars, and foods with trans fats (“Mediterranean Diet 101”. According to “Mediterranean Diet: Ranked #1,...” by health.us.news, the Mediterranean diet is ranked #1 Best Diet Overall, #1 Best Diabetes Diet, #1 Best Weight-Loss Diets, #1 Best Diets for Healthy Eating, and #1 in Easiest Diets to Follow. One downside to the Mediterranean Diet is that it is less cost-friendly than other diets. This issue is especially relevant to our topic of interest: minimum cost diet. 

<i> Cal Dining and Bear Fit  </i>

Cal Dining is a part of Housing and Dining Operations, Residential and Student Service Programs (RSSP) here at the University of California, Berkeley. There are multiple meal plans for students living both on and off campus. For students, these plans are valid for the full academic year. Meal plans include a combination of meal swipes and flex dollars. Meal swipes are for use in the dining commons or as a “meal equivalency” swipe at various cafes on campus. Flex dollars may be used at campus restaurants, convenience stores, concession stands, and to pay for guest meals.
Flex dollars function like cash dollars—1 flex dollar has the purchasing power of 1 dollar. For example, a purchase of $5 would cost 5 flex dollars. 

Our csv and code utilize data provided from the Cal Dining website. We chose four “Bear Fit” meals from the Crossroads Dining Hall, which are considered to be the healthiest options. Bear Fit meals is designed to serve nutritious and delicious food that everyone can enjoy without worrying about common allergic or triggering ingredients with a menu free of the Big-8 food allergens: sesame, gluten, pork, and alcohol. From these particular meals, we noted various nutritional facts in the ingredients used such as energy, protein, iron, and more, compared to those in the Mediterranean diet.

This information about Cal Dining  is relevant to our project as we consider and compare the various options that students have when choosing how they will eat for the academic year, particularly if the student wishes to maintain a healthy diet at the lowest financial cost.


### <u> Coding Deliverables </u>

Setting up the environment:

In [1]:
!pip install -r requirements.txt

  Using cached Pint-0.18-py2.py3-none-any.whl (209 kB)
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
Processing /home/jovyan/.cache/pip/wheels/20/7e/30/7d702acd6a1e89911301cd9dbf9cb9870ca80c0e64bc2cde23/gnupg-2.3.1-py3-none-any.whl
  Attempting uninstall: pint
    Found existing installation: Pint 0.17
    Uninstalling Pint-0.17:
      Successfully uninstalled Pint-0.17


In [6]:
import numpy as np
import pandas as pd
import http.client
import fooddatacentral as fdc

In [7]:
apikey = "an9JzugedN7gb0EvKlgsVs2iMVLIFVYOLcpdqUqo"  # Replace with a real key!  "DEMO_KEY" will be slow...

#### [A] Dietary Reference Intakes

We divided the total daily intake by 3 for all desired values under the assumption that there are about 3 meals per day that thee average male and female have. We also assume that the quantity of food, and hence the nutritional value of each meal, is roughly the same. This is useful because we can directly match up our nutritional requirements for one meal to our minimal cost diet, which also reflects one meal. 

In [10]:
diet_req = pd.read_csv("dietary_req.csv").drop(['Source'],axis=1)
diet_req = diet_req.set_index('Nutrition')
dietreq_meal=diet_req.div(3)
dietreq_meal.iloc[len(dietreq_meal)-1,:] = dietreq_meal.iloc[len(dietreq_meal)-1,:] * -1

In [11]:
groups = ['M 19-30','F 19-30']
def create_pop(group,activity):
    pop_req = dietreq_meal[group]
    if activity:
        pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300
    return pop_req
group = create_pop(groups, True).drop(0,axis=0)
group

/tmp/ipykernel_128/313555015.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300


,M 19-30,F 19-30
Nutrition,,
Energy,800.000000,666.666667
Protein,18.666667,15.333333
"Fiber, total dietary",11.200000,9.333333
"Folate, DFE",133.333333,133.333333
"Calcium, Ca",333.333333,333.333333
"Carbohydrate, by difference",43.333333,43.333333
"Iron, Fe",2.666667,6.000000
"Magnesium, Mg",133.333333,103.333333
Niacin,5.333333,4.666667


#### [A] Nutritional Content of Different Foods

In [17]:
price_nutrition = pd.read_csv('PriceData.csv')
price_nutrition=price_nutrition.set_index("Ingredients").drop(['Meal Name','Size','ID','fdcId','Brand'],axis=1)
price_nutrition[0:4]

,Energy,Protein,"Fiber, total dietary","Folate, DFE","Calcium, Ca","Carbohydrate, by difference","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P",...,Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn","Sodium, Na",Price
Ingredients,,,,,,,,,,,,,,,,,,,,,
Asparagus,18.0,2.35,2.4,52.0,24.0,3.53,2.12,14.0,0.98,52.0,...,0.143,38,0.0,0.091,5.9,1.13,41.6,0.54,2,1.88
Avacado Chunks,160.0,2.00,6.7,81.0,12.0,8.53,0.55,29.0,1.74,52.0,...,0.067,7,0.0,0.257,10.0,2.07,21.0,0.64,7,2.50
Flavored Brown Rice,106.0,2.76,1.2,10.0,7.0,19.30,0.39,37.0,1.68,89.0,...,0.071,16,0.0,0.106,0.0,0.35,2.4,0.99,293,6.00
Carrots,38.0,1.28,2.6,19.0,38.0,8.97,0.26,12.0,0.98,35.0,...,0.066,835,0.0,0.138,6.4,0.66,13.2,0.24,71,1.00


#### [A] Data on Prices for Different Foods  

In [16]:
price_nutrition['Price']

Ingredients
Asparagus                 1.88
Avacado Chunks            2.50
Flavored Brown Rice       6.00
Carrots                   1.00
Chicken                   8.00
Chickpeas                 2.00
Greek Yogurt              6.00
Pear                      2.00
White Potatoes            3.00
Raspberries               7.00
Salmon                   13.00
Strawberries              5.00
Unsalted Almonds         11.00
Walnuts                   8.00
White Beans               1.67
Whole Wheat Pita          3.50
Black Peas                3.00
Black Rice               12.00
Carrots                   1.00
Cilantro                  1.30
Jalapeno Peppers          1.00
Lemon Juice               0.75
Lime Juice                0.60
Pepper Bell Red Whole     2.00
Shiitake Mushroom         6.00
Sugar                     4.30
Vinegar                   4.00
Black Pepper              3.50
Halal Ground Beef        10.50
Parsley                   1.00
Salt                      1.50
Halal Chicken             8

In [13]:
from  scipy.optimize import linprog as lp
def solve_subsistence_problem(FoodNutrients,nutrients, prices):
    A = price_nutrition[nutrients].apply(pd.to_numeric).T
    b = dietreq_meal.iloc[0,:]
    result = lp(prices, A, -b, method='interior-point')
    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  
    return result

In [14]:
Prices = price_nutrition[np.append(group.index,'Price')]['Price'].to_list()
Prices = [float(s) for s in Prices] 
result = solve_subsistence_problem(price_nutrition,group.index, Prices)

ValueError: Invalid input for linprog: b_ub must be a 1-D array; b_ub must not have more than one non-singleton dimension and the number of rows in A_ub must equal the number of values in b_ub

Note: We are having a bit of trouble debugging the lp function so we could definitely use your suggestions on how to fix this! We're going to debug this in class on Monday with Professor Ligon so we can find our minimum cost diet. 

<b> [B] What is the total cost for the population of interest? </b>

Our total population is the total population of undergraduate students at UC Berkeley. According to US News, UC Berkeley has about a total undergraduate population of 31,814 students and 13,243 graduate students. The male:female ratio for both types of students 17,226:14,321 and 6,321:6,762 respectively. Most of the students in both degrees are between the ages of 19-30, which all fall into the age range bucket of 19-30 in our data set. We will update this with the total cost when we finish calculating the min cost diet. 

Source: https://opa.berkeley.edu/campus-data/uc-berkeley-quick-facts